In [36]:
import Model
import tensorflow as tf
import os
import re
import hashlib
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

from extractMFCC import computeFeatures, computeFeatures1
from addNoise import addNoise
print(tf.__version__)

2.0.0-beta0


In [37]:
testWAVs = pd.read_csv('Dataset/testing_list.txt', sep=" ", header=None)[0].tolist()
valWAVs  = pd.read_csv('Dataset/validation_list.txt', sep=" ", header=None)[0].tolist()

for idx, item in enumerate(testWAVs):
    testWAVs[idx] = "Dataset/" + testWAVs[idx]

for idx, item in enumerate(valWAVs):
    valWAVs[idx] = "Dataset/" + valWAVs[idx]

In [38]:
DictCategs = {'nine' : 1, 'yes' : 2, 
                         'no' : 3, 'up' : 4, 'down' : 5, 'left' : 6, 'right' : 7, 'on' : 8, 'off' : 9, 'stop' : 10, 'go' : 11,
                         'zero' : 12, 'one' : 13, 'two' : 14, 'three' : 15, 'four' : 16, 'five' : 17, 'six' : 18, 
                         'seven' : 19,  'eight' : 20, 'backward':0, 'bed':0, 'bird':0, 'cat':0, 'dog':0,
                         'follow':0, 'forward':0, 'happy':0, 'house':0, 'learn':0, 'marvin':0, 'sheila':0, 'tree':0,
                         'visual':0, 'wow':0}
nCategs = 21

In [39]:
allWAVs  = []
for root, dirs, files in os.walk('Dataset/'):
    if root != "Dataset/_background_noise_":
        allWAVs += [root+'/'+ f for f in files if f.endswith('.wav')]
trainWAVs = list( set(allWAVs)-set(valWAVs)-set(testWAVs) )

In [40]:
print(len(trainWAVs))
print(len(valWAVs))
print(len(testWAVs))

84843
9981
11005


In [41]:
def _getFileCategory(file, catDict):
    """
    Receives a file with name Dataset/<cat>/<filename> and returns an integer that is catDict[cat]
    """
    categ = os.path.basename(os.path.dirname(file))
    return catDict.get(categ,0)

In [42]:
#get categories
testWAVlabels = [_getFileCategory(f, DictCategs) for f in testWAVs]
valWAVlabels = [_getFileCategory(f, DictCategs) for f in valWAVs]
trainWAVlabels = [_getFileCategory(f, DictCategs) for f in trainWAVs]

In [43]:
print(len(trainWAVlabels))
print(len(valWAVlabels))
print(len(testWAVlabels))
#print(trainWAVs[:20])

84843
9981
11005


In [44]:
import scipy
from scipy.io.wavfile import read
import python_speech_features as sf
import numpy as np
from addNoise import addNoise2

def cp(wav_signal_name, desiredLength = 16000, log = True, w_len = 0.025, w_step = 0.01, noise = "dude_miaowing.wav"):
    # input:   a wav audio file (ours are all 1 second long, so 16000 sample long if Fc = 16000)
    # the noise is the one to add to the signal if the signal is not full length
    # output:  a 2D matrix of size (num_frames, 39), where 39 is the number of coefficients of the features vectors
    # data are supposed to be single channel
    # log = True: it means that we take the logarithm of the energies of delta and delta-delta
    
    # read audio sample
    print(wav_signal_name)
    input_signal = read(wav_signal_name)
    # sampling rate
    
    Fc = input_signal[0]
    # Nyquist critical frequency(highest freq. that can be represented)
    nyqF = int(Fc/2)  #input in sf.base.mfcc() has to be an int
    signal = input_signal[1] # this is the vector-signal we are interested in
    if(signal.shape[0] < desiredLength):
        noise = read(noise)[1]
        signal = addNoise2(signal, noise, begin = True)
    # appendEnergy = True means that the zeroth value of all the cepstral vectors is replaced  with the corresponding frame energy, E(s_i)
    coeffs = sf.base.mfcc(signal, samplerate = Fc, nfft = nyqF, appendEnergy = True, winlen=w_len, winstep=w_step, winfunc=np.hamming)
    useful_coeffs = coeffs[:,1:13] # (taking only the 1,2,...,12 MFCC's)
    # energy(s)
    E_s = coeffs[:,0]
    # take the log10 to limit the dynamics
    log_E_s = np.log10(E_s)
    #print("shape of log energies of the frames: " + str(log_E_s.shape))
    #print("shape of useful coefficients: " + str(useful_coeffs.shape))

    # extracting deltas
    deltas = sf.base.delta(useful_coeffs, 2)
    #print("deltas = " + str(deltas.shape))
    
    # extracting deltas of deltas
    deltas_2 = sf.base.delta(deltas, 2)
    #print("deltas_2 = " + str(deltas_2.shape))
    
    # getting energies of deltas and of deltas_2
    E_deltas = np.sum(np.power(deltas, 2), axis = 1)
    log_E_deltas = np.log10(E_deltas)
    #print("shape of log_E_deltas = " + str(log_E_deltas.shape))
    
    # energy of delta of deltas: 
    E_deltas_2 = np.sum(np.power(deltas, 2), axis = 1)
    log_E_deltas_2 = np.log10(E_deltas_2)
    #print("shape of log_E_deltas_2 = " + str(log_E_deltas_2.shape))

    if(log):
        E_d = log_E_deltas
        E_d_d = log_E_deltas_2
    else:
        E_d = E_deltas    
        E_d_d = E_deltas_2

    # preallocating space:
    num_frames = coeffs.shape[0]
    #num_frames = 99
    features = np.zeros((num_frames,39))

    # putting everything inside the features 2D matrix:
    features[0:coeffs.shape[0], 0:12] = useful_coeffs
    features[0:coeffs.shape[0], 12:24] = deltas
    features[0:coeffs.shape[0], 24:36] = deltas_2
    features[0:coeffs.shape[0], 36] = log_E_s
    features[0:coeffs.shape[0], 37] = E_d
    features[0:coeffs.shape[0], 38] = E_d_d
    
    return features


In [61]:
def load_and_preprocess_data(file_name):
    # Required by tensorflow (strings are passed as bytes)
    file_name = tf.io.read_file(file_name)
    file_name = file_name.eval()
    #if type(file_name) is bytes:
    #    file_name = file_name.decode()
    print(file_name)
    # Load data
    feats = cp(file_name)
    # Normalize
    mean = np.mean(feats, axis = 0)
    stv = np.std(feats, axis = 0)
    diff = np.subtract(feats, mean)
    feats = np.divide(diff, stv)

    return feats.astype(np.float32)
"""
# example:
feats = load_and_preprocess_data(trainWAVs[25])
feats = np.transpose(feats)

plt.figure(figsize=(17,6))
plt.pcolormesh(feats)

plt.title('Spectrogram visualization')
plt.ylabel('Frequency')
plt.xlabel('Time')

plt.show()
print(trainWAVlabels[25])"""

"\n# example:\nfeats = load_and_preprocess_data(trainWAVs[25])\nfeats = np.transpose(feats)\n\nplt.figure(figsize=(17,6))\nplt.pcolormesh(feats)\n\nplt.title('Spectrogram visualization')\nplt.ylabel('Frequency')\nplt.xlabel('Time')\n\nplt.show()\nprint(trainWAVlabels[25])"

In [62]:
path_ds = tf.data.Dataset.from_tensor_slices(trainWAVs)

In [63]:
image_ds = path_ds.map(load_and_preprocess_data, num_parallel_calls=os.cpu_count())

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [49]:
def create_dataset(file_names, labels, batch_size, shuffle, cache_file=None):
    
    # Create a Dataset object
    dataset = tf.data.Dataset.from_tensor_slices(file_names)
    
    # Map the load_and_preprocess_data function
    #py_func = lambda file_name, label: (tf.py_function(load_and_preprocess_data, [file_name], tf.float32), label)
    dataset = dataset.map(load_and_preprocess_data, num_parallel_calls=os.cpu_count())
    
    # Cache dataset
    if cache_file:
        dataset = dataset.cache(cache_file)
    
    # Shuffle    
    if shuffle:
        dataset = dataset.shuffle(len(file_names))
        
    # Repeat the dataset indefinitely
    dataset = dataset.repeat()
    
    # Correct input shape for the network
    dataset = dataset.map(lambda data, label: (tf.expand_dims(data, 1), label))
    
    # Batch
    dataset = dataset.batch(batch_size=batch_size)
    
    # Prefetch
    dataset = dataset.prefetch(buffer_size=1)
    
    return dataset

In [50]:
#train = tf.constant(trainWAVs)
#train_labels = tf.constant(trainWAVlabels)

batch_size = 32
train_dataset = create_dataset(trainWAVs, 
                               trainWAVlabels, 
                               batch_size=batch_size, 
                               shuffle=True,
                               cache_file='train_cache')

val_dataset = create_dataset(valWAVs, 
                             valWAVlabels,
                             batch_size=batch_size, 
                             shuffle=False,
                             cache_file='val_cache')

train_steps = int(np.ceil(len(trainWAVs)/batch_size))
val_steps = int(np.ceil(len(valWAVs)/batch_size))

Tensor("ReadFile:0", shape=(), dtype=string)
Tensor("ReadFile:0", shape=(), dtype=string)


TypeError: invalid file: <tf.Tensor 'ReadFile:0' shape=() dtype=string>

In [ ]:
model = Model.AttentionModel(21, 99, 39, use_GRU = True)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [130]:
num_epochs = 30
history = model.fit(train_dataset, epochs=num_epochs, steps_per_epoch=train_steps, validation_data=val_dataset, validation_steps=val_steps)

# Save the model
model.save('my_model.h5')

# Plot loss
plt.figure()
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Val loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Plot accuracy
plt.figure()
plt.plot(history.history['acc'], label='Train loss')
plt.plot(history.history['val_acc'], label='Val loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

Epoch 1/30
tf.Tensor(b'Dataset/wow/f15a354c_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/wow/f15a354c_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/five/cd7f8c1b_nohash_4.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/five/cd7f8c1b_nohash_4.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/go/4ec7d027_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/go/4ec7d027_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/zero/c38720cb_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/zero/c38720cb_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/left/3a3ee7ed_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/left/3a3ee7ed_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/zero/91ffb786_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/zero/91ffb786_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/nine/1dc86f91_nohash_2.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/nine/1dc86f91_nohash_2.wav

InvalidArgumentError: TypeError: Can't convert 'tensorflow.python.framework.ops.EagerTensor' object to str implicitly
Traceback (most recent call last):

  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/script_ops.py", line 207, in __call__
    return func(device, token, args)

  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/script_ops.py", line 109, in __call__
    ret = self._func(*args)

  File "<ipython-input-125-2557e5fca52a>", line 8, in load_and_preprocess_data
    feats = cp(file_name)

  File "<ipython-input-84-aced9627f1ba>", line 16, in cp
    input_signal = read(wav_signal_name)

  File "/usr/local/lib/python3.5/site-packages/scipy/io/wavfile.py", line 233, in read
    fid = open(filename, 'rb')

TypeError: Can't convert 'tensorflow.python.framework.ops.EagerTensor' object to str implicitly


	 [[{{node EagerPyFunc}}]] [Op:IteratorGetNextSync]

tf.Tensor(b'Dataset/follow/cdee383b_nohash_1.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/follow/cdee383b_nohash_1.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/on/4422e51d_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/on/4422e51d_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/stop/e882abb2_nohash_3.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/stop/e882abb2_nohash_3.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/tree/c1eebc0b_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/tree/c1eebc0b_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/go/821b64cc_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/go/821b64cc_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/left/ccb1266b_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/left/ccb1266b_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/off/d430b3cc_nohash_0.wav', shape=(), dtype=string)
tf.Tensor(b'Dataset/off/d430b3cc_nohash_0.wav', shape=()